In [526]:
import scipy.io
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [546]:
def cofiCostFunc(param, Y, R, num_users, num_movies, num_features, lambda_):    
    X = np.reshape(param[0:(num_movies*num_features)], (num_movies, num_features));
    Theta = np.reshape(param[(num_movies*num_features):], (num_users, num_features));
    J = (sum(sum(((X@(Theta.T) - Y)*R)**2))/2) + ((lambda_/2) * (sum(sum(Theta**2)) + sum(sum(X**2))));
    
    X_grad = np.zeros((X.shape));
    Theta_grad = np.zeros((Theta.shape));
    
    for i in range(num_movies):
        indexes = np.where(R[i,:] == 1)[0];
        if (len(indexes) > 0):
            theta_temp = Theta[indexes, :];
            Y_temp = Y[i, indexes];
            X_grad[i, :] = (((X[i, :] @ theta_temp.T) - Y_temp) @ theta_temp) + (lambda_ * X[i, :]);
    
    for j in range(num_users):
        indexes = np.where(R[:, j] == 1)[0];
        if (len(indexes) > 0):
            X_temp = X[indexes, :];
            Y_temp = Y[indexes, j];
            Theta_grad[j, :] = (((X_temp @ Theta[j, :].T) - Y_temp).reshape(-1,1).T @ X_temp) + (lambda_ * Theta[j, :]);
    
    grad = np.concatenate((X_grad.flatten('F'), Theta_grad.flatten('F')));
    
    return J, grad;


In [547]:
def normalizeRatings(Y, R):    
    m, n = Y.shape;
    
    Y_mean = np.zeros((m, 1));
    Y_norm = np.zeros((Y.shape));
    
    for i in range(m):
        indexes = np.where(R[i,:] == 1)[0];
        Y_mean[i] = np.mean(Y[i, indexes]);
        Y_norm[i, indexes] = Y[i, indexes] - Y_mean[i];
        
    return Y_norm, Y_mean;

In [548]:
mat_data = scipy.io.loadmat('ex8_movies.mat');

Y = np.array(pd.DataFrame(mat_data['Y']));
R = np.array(pd.DataFrame(mat_data['R']));

mat_param = scipy.io.loadmat('ex8_movieParams.mat');

X = np.array(pd.DataFrame(mat_param['X']));
Theta = np.array(pd.DataFrame(mat_param['Theta']));

num_users = int(mat_param['num_users']);
num_movies = int(mat_param['num_movies']);
num_features = int(mat_param['num_features']);

In [549]:
# manual test cost function
num_f = 3;
num_m = 5;
num_u = 4;

X_test = X[0:num_m, 0:num_f];
Theta_test = Theta[0:num_u, 0:num_f];

Y_test = Y[0:num_m, 0:num_u];
R_test = R[0:num_m, 0:num_u];

np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
param_vec = np.concatenate((X_test.flatten(), Theta_test.flatten())).reshape(-1,1);

# With regularisation parameter = 0
J, grad = cofiCostFunc(param_vec, Y_test, R_test, num_u, num_m, num_f, 0);

print(J)

22.22460372568567


(this value should be about 22.22)

In [550]:
print(grad)

[ -2.52899165  -0.56819597  -0.83240713  -0.38358278  -0.80378006
   7.57570308   3.35265031   4.91163297   2.26333698   4.74271842
  -1.89979026  -0.52339845  -0.76677878  -0.35334048  -0.74040871
 -10.5680202   -3.05099006   0.           0.           4.62776019
   1.16441367   0.           0.          -7.16004443  -3.47410789
   0.           0.        ]


Expected Value: [-2.5290
   -0.5682
   -0.8324
   -0.3836
   -0.8038
    7.5757
    3.3527
    4.9116
    2.2633
    4.7427
   -1.8998
   -0.5234
   -0.7668
   -0.3533
   -0.7404
  -10.5680
   -3.0510
         0
         0
    4.6278
    1.1644
         0
         0
   -7.1600
   -3.4741
         0
         0]

In [551]:
# With regularisation parameter = 1.5
J, grad = cofiCostFunc(param_vec, Y_test, R_test, num_u, num_m, num_f, 1.5);

print(J)

31.344056244274213


(this value should be about 31.34)

In [557]:
# Colaborative filtering
my_ratings = np.zeros((1682, 1));

my_ratings[1] = 4;
my_ratings[98] = 2;
my_ratings[7] = 3;
my_ratings[12]= 5;
my_ratings[54] = 4;
my_ratings[64]= 5;
my_ratings[66]= 3;
my_ratings[69] = 5;
my_ratings[183] = 4;
my_ratings[226] = 5;
my_ratings[355]= 5;

Y_new = np.hstack((my_ratings, Y));
R_new = np.hstack((np.where(my_ratings != 0, 1, 0), R));

Ynorm, Ymean = normalizeRatings(Y_new, R_new);

num_users_new = Y_new.shape[1];

# Set initial Parameters.
X = np.random.randn(num_movies, num_features);
Theta = np.random.randn(num_users_new, num_features);

print(X);
print(Theta);

initial_params = np.concatenate((X.flatten(), Theta.flatten())).reshape(-1,1);

print(initial_params);

[[-0.37019531 -1.07088403  0.13738192 ...  1.10142645 -1.32763038
   0.0826835 ]
 [ 0.36930243  0.10185128 -0.136252   ... -0.08150032  1.38042063
  -0.4219028 ]
 [ 0.27548158 -0.31067409 -1.4922189  ... -0.78412407  1.42928406
   0.85088489]
 ...
 [-0.84949038  0.78403124 -0.18432336 ...  1.61398286  1.36338924
   0.32177806]
 [ 0.10342561  1.80015483 -1.25445081 ...  0.29984012 -0.03332759
  -0.24713014]
 [ 0.5918988  -0.38999149 -1.09204119 ...  1.87287832 -0.69424057
  -0.34875914]]
[[ 1.23935066  1.54477282 -0.52177669 ...  0.10231996 -0.92709367
   0.594863  ]
 [-1.71110986 -1.32963354 -0.78649517 ...  1.54724999  0.6560323
   0.70369735]
 [ 0.25001086 -0.26092745 -0.018085   ... -1.0089822  -0.96971716
  -0.58969276]
 ...
 [-1.0507446  -1.13712594  1.91265636 ...  1.48163988 -0.24286889
   0.98988386]
 [ 0.32761376 -1.53603193  0.69901903 ...  1.21555976  0.87109516
   1.03236019]
 [ 1.82978881 -1.17746676 -0.07213889 ...  0.05993706 -0.0039902
   0.36991598]]
[[-0.37019531]
 [-

In [559]:
lambda_ = 10;

options= {'maxiter': 100}

res = minimize(cofiCostFunc, 
               initial_params, 
               args = (Ynorm, R_new, num_users_new, num_movies, num_features, lambda_), 
               jac=True,
               method='CG',
               options=options);

print(res.fun)

664484.597776093
